In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable
import astropy.units as u
import sys
import pyneb as pn
from astropy.io import fits
import aplpy
from matplotlib import gridspec
from astropy.io import ascii 
import scipy
from scipy.optimize import curve_fit
from scipy.integrate import quad
import numpy.linalg as la
import pyneb as pn
from astropy.io import fits
import aplpy
from matplotlib import gridspec

### Import initial data, Temperatures and Densities

In [ ]:
infile = open("/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/temdendata.fits",'rb')
data = Table.read(infile)

### Import Refitted data

In [ ]:
infile = open("/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/oiirefit.fits",'rb')
oiidata = Table.read(infile)

In [ ]:
oiidata

### PyNeb Diagnostic Information

In [ ]:
diags = pn.Diagnostics()

icf = pn.ICF()
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
icf_list = ['Ial06_16'] #'direct_O.23','KB94_A10b','Ial06_16'

In [ ]:
count = 0
for i in range(len(data)):
    if np.isnan(data[i]['OIII_temp']) == False and np.isnan(data[i]['NII_temp']) == False and np.isnan(oiidata[i]['OII7319_FLUX_CORR_REFIT']) == False and np.isnan(oiidata[i]['OII7330_FLUX_CORR_REFIT']) == False and np.isnan(data[i]['OIII_temp']) == False:
        count += 1
count

### Metallicity Function

In [ ]:
def metal(indata, oii, err):
  brightoii = oii[[i for i in range(len(indata)) if np.isnan(oii[i]['OII7319_FLUX_CORR_REFIT']) == False and 
                   indata[i]['OIII5006_FLUX_CORR'] > indata[i]['OIII5006_FLUX_CORR_ERR'] * err and 
                   np.isnan(indata[i]['NII_temp']) == False and np.isnan(indata[i]['SII_den_NII']) == False and 
                   np.isnan(indata[i]['OIII_temp']) == False]]
  brightreg = indata[[i for i in range(len(indata)) if np.isnan(oii[i]['OII7319_FLUX_CORR_REFIT']) == False and 
                      indata[i]['OIII5006_FLUX_CORR'] > indata[i]['OIII5006_FLUX_CORR_ERR'] * err and 
                      np.isnan(indata[i]['NII_temp']) == False and np.isnan(indata[i]['SII_den_NII']) == False and 
                      np.isnan(indata[i]['OIII_temp']) == False]]
  metalnums = np.array([brightreg[i]['REGION_ID'] for i in range(len(brightreg))])
  metalnums2 = np.array([brightreg[i]['REGION_ID'] for i in range(len(brightreg)) if np.isnan(oii[i]['OII7330_FLUX_CORR_REFIT']) == False])

  OI6300 = O1.getIonAbundance(int_ratio = brightreg['OI6300_FLUX_CORR'], tem = brightreg['NII_temp'], 
                              den= brightreg['SII_den_NII'], wave = 6300, Hbeta = brightreg['HB4861_FLUX_CORR'])
  OI6363 = O1.getIonAbundance(int_ratio = brightreg['OI6363_FLUX_CORR'], tem = brightreg['NII_temp'], 
                              den= brightreg['SII_den_NII'], wave = 6363, Hbeta = brightreg['HB4861_FLUX_CORR'])
  OII7319 = O2.getIonAbundance(int_ratio = brightoii['OII7319_FLUX_CORR_REFIT'], tem = brightreg['NII_temp'], 
                               den= brightreg['SII_den_NII'], wave = 7319, Hbeta = brightreg['HB4861_FLUX_CORR'])
  OII7330 = O2.getIonAbundance(int_ratio = brightoii['OII7330_FLUX_CORR_REFIT'], tem = brightreg['NII_temp'], 
                               den= brightreg['SII_den_NII'], wave = 7330, Hbeta = brightreg['HB4861_FLUX_CORR'])
  OIII5006 = O3.getIonAbundance(int_ratio = brightreg['OIII5006_FLUX_CORR'], tem = brightreg['OIII_temp'], 
                                den= brightreg['SII_den_SIII'], wave = 5007, Hbeta = brightreg['HB4861_FLUX_CORR'])
  
  #abunlist = {'O1': OI6300, 'O1': OI6363, 'O2': OII7319, 'O2': OII7330, 'O3': OIII5006}
  abunlist = {'O2': OII7319, 'O3': OIII5006}
  abunlist2 = {'O2': OII7330, 'O3': OIII5006}
  #abunlist = {'O2': OII7319}

  abundance = icf.getElemAbundance(abunlist, icf_list)
  mets = 12 + np.log10(abundance['Ial06_16'])
  ometal = np.array([mets[i] for i in range(len(mets))])
    
  abundance2 = icf.getElemAbundance(abunlist2, icf_list)
  mets2 = 12 + np.log10(abundance2['Ial06_16'])
  ometal2 = np.array([mets2[i] for i in range(len(mets2))])

  fmetal = np.zeros(len(indata))
  fmetal2 = np.zeros(len(indata))
  oiimetal = np.zeros(len(indata))
  oiiimetal = np.zeros(len(indata))
  fmetal.fill('nan')
  fmetal2.fill('nan')
  oiimetal.fill('nan')
  oiiimetal.fill('nan')
  metalcount = 0
  metalcount2 = 0

  for i in range(len(indata)):
    if indata[i]['REGION_ID'] in metalnums:
      fmetal[i] = ometal[metalcount]
      oiimetal[i] = OII7319[metalcount]
      oiiimetal[i] = OIII5006[metalcount]
      metalcount += 1
    if indata[i]['REGION_ID'] in metalnums2:
      fmetal2[i] = ometal2[metalcount2]
      metalcount2 += 1
  print(metalcount, metalcount2)

  indata.add_columns([fmetal, fmetal2, oiimetal, oiiimetal], names=('met_derived_OII7319','met_derived_OII7330', 'OII_ABUN', 'OIII_ABUN'))

  return indata

In [ ]:
metdata = metal(data, oiidata, 0)

### Strong line and directly derived metallicities

In [ ]:
dngc4254 = 13.1 
kpc = data['DEPROJ_DIST'] / 206265 * (dngc4254 * 1000)
nkpc = np.array([kpc[i] for i in range(len(kpc))])
seq = np.linspace(-3, 15, num = 20)

In [ ]:
dirc = np.array([metdata[i]['met_derived_OII7319'] for i in range(len(metdata)) if np.isnan(metdata[i]['met_derived_OII7319']) == False])
direct = np.polyfit(nkpc[np.where(np.isnan(metdata['met_derived_OII7319']) == False)], dirc, 1)
dirfunc = np.poly1d(direct)
np.std(dirc)

In [ ]:
s = np.array([metdata[i]['MET_SCAL'] for i in range(len(metdata)) if np.isnan(metdata[i]['MET_SCAL']) == False])
scal = np.polyfit(nkpc[np.where(np.isnan(metdata['MET_SCAL']) == False)], s, 1)
scalfunc = np.poly1d(scal)
np.std(s)

In [ ]:
fig = plt.figure()
fig.set_size_inches(6, 4)
ax1 = fig.add_subplot(111)

ax1.scatter(kpc, metdata['MET_SCAL'], s=10, c='orange', marker= '.', label='Scal-PG16')
#ax1.scatter(kpc, metdata['met_derived_OII7319'], s=20, c='red', marker= '.', label='Directly Derived from [O II]7319')
#ax1.scatter(kpc, metdata['met_derived_OII7330'], s=20, c='blue', marker= '.', label='Directly Derived from [O II]7330')
#ax1.plot(seq, dirfunc(seq), c='k', linestyle="--", label=f'Directly Derived: y = {round(dirfunc[1],3)}x + {round(dirfunc[0],3)}')
ax1.plot(seq, scalfunc(seq), c='k', label=f'Scal-PG16: y = {round(scalfunc[1],3)}x + {round(scalfunc[0],3)}')

minm = np.min(metdata['met_derived_OII7319'][np.where(np.isnan(metdata['met_derived_OII7319']) == False)])
maxm = np.max(metdata['met_derived_OII7319'][np.where(np.isnan(metdata['met_derived_OII7319']) == False)])
plt.xlim([0, 12])
#plt.ylim([minm - 0.065*minm, maxm + 0.005*maxm])

plt.xlabel('Galactocentric Radius (kpc)', fontsize=15)
plt.ylabel('12 + log(O/H)', fontsize=15)
plt.legend(fontsize="8",loc='lower left');
plt.show()

### Refit [OII]7319 S/N versus metallicity

In [ ]:
fig = plt.figure()
fig.set_size_inches(6, 4)
ax1 = fig.add_subplot(111)

ax1.scatter(oiidata['SNOII7319'], metdata['met_derived_OII7319'], s=20, c='red', marker= '.', label='Directly Derived')

#plt.xlim([0, 12])

minm = np.min(metdata['met_derived_OII7319'][np.where(np.isnan(metdata['met_derived_OII7319']) == False)])
maxm = np.max(metdata['met_derived_OII7319'][np.where(np.isnan(metdata['met_derived_OII7319']) == False)])
#plt.xlim([minm - 0.05*minm, maxm + 0.05*maxm])
plt.xlabel('[OII]7319 Refit S/N')
plt.ylabel('12 + log(O/H)')
#plt.legend(loc='best');
plt.show()

In [ ]:
np.where(np.isnan(metdata['met_derived_OII7319']) == False)

In [ ]:
oiidata[np.where(np.isnan(metdata['met_derived_OII7319']) == False)]

In [ ]:
fig = plt.figure()
fig.set_size_inches(6, 4)
ax1 = fig.add_subplot(111)

ax1.scatter(kpc, np.log(metdata['OII_ABUN']/metdata['OIII_ABUN']), s=20, c='purple', marker= '.', label='log([O II] / [O III])')

abun = np.array([np.log(metdata[i]['OII_ABUN']/metdata[i]['OIII_ABUN']) for i in range(len(metdata)) if np.isnan(metdata[i]['OII_ABUN']) == False])
abuns = np.polyfit(nkpc[np.where(np.isnan(np.log(metdata['OII_ABUN']/metdata['OIII_ABUN'])) == False)], abun, 1)
abunfunc = np.poly1d(abuns)

ax1.plot(seq, abunfunc(seq), c='k', label=f'Fit Line: y = {round(abunfunc[1],3)}x + {round(abunfunc[0],3)}')

plt.xlim([0, 12])
plt.ylim([1.7, 7.7])

plt.xlabel('Galactocentric Radius (kpc)')
plt.ylabel('log([O II] / [O III])')
plt.legend(fontsize="8",loc='lower left');
plt.show()

In [ ]:
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}
galaxy = galdic[1]
infile = open(f"/Users/erich/Downloads/Undergraduate-Research/HII-Region-Jupyter-Notebooks/{galaxy}_refitdata.fits",'rb')
data = Table.read(infile)
galaxy

In [ ]:
data = data[np.array([i for i in range(len(data)) if np.isnan(data[i]['OII7319_FLUX_CORR_REFIT']) == False and 
                      np.isnan(data[i]['OII7330_FLUX_CORR_REFIT']) == False and
                     np.isnan(data[i]['SIII6312_FLUX_CORR_REFIT']) == False])]

data[:5]

In [ ]:
diags = pn.Diagnostics()

def niitemperr(indata, iters):
    
    temperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise5754 = np.random.normal(0, indata['NII5754_FLUX_CORR_REFIT_ERR'])
        noise6583 = np.random.normal(0, indata['NII6583_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        temperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['NII5754_FLUX_CORR_REFIT']+noise5754)/(indata['NII6583_FLUX_CORR']+noise6583), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
            
    temperr = np.array([temperr[i] for i in range(len(temperr)) if np.isnan(temperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(temperr), np.std(denerr)

def siiitemperr(indata, iters):
    
    temperr = np.zeros(iters)
    denerr = np.zeros(iters)
    
    for j in range(iters):
        noise6312 = np.random.normal(0, indata['SIII6312_FLUX_CORR_REFIT_ERR'])
        noise9069 = np.random.normal(0, indata['SIII9068_FLUX_CORR_ERR'])
        noise6730 = np.random.normal(0, indata['SII6730_FLUX_CORR_ERR'])
        noise6716 = np.random.normal(0, indata['SII6716_FLUX_CORR_ERR'])

        temperr[j], denerr[j] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                                     value_tem = (indata['SIII6312_FLUX_CORR_REFIT']+noise6312)/(indata['SIII9068_FLUX_CORR']+noise9069), 
                                                     value_den = (indata['SII6730_FLUX_CORR']+noise6730)/(indata['SII6716_FLUX_CORR']+noise6716), 
                                                     guess_tem = 10**4)
            
    temperr = np.array([temperr[i] for i in range(len(temperr)) if np.isnan(temperr[i]) == False])
    denerr = np.array([denerr[i] for i in range(len(denerr)) if np.isnan(denerr[i]) == False])
    
    return np.std(temperr), np.std(denerr)

In [ ]:
def dentemp(indata, err, erriter):
  
  tenii = np.zeros(len(indata))
  nenii = np.zeros(len(indata))
  teniierr = np.zeros(len(indata))
  neniierr = np.zeros(len(indata))
  for i in range(len(indata)):
        if indata[i]['BPT_NII'] == 0 and indata[i]['NII6583_FLUX_CORR'] > err * indata[i]['NII6583_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['NII5754_FLUX_CORR_REFIT'] > err * indata[i]['NII5754_FLUX_CORR_REFIT_ERR']:
            tenii[i], nenii[i] = diags.getCrossTemDen(diag_tem = '[NII] 5755/6584', diag_den = '[SII] 6731/6716', 
                                      value_tem = indata[i]['NII5754_FLUX_CORR_REFIT']/indata[i]['NII6583_FLUX_CORR'], 
                                                value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                      guess_tem = 10**4)
            teniierr[i], neniierr[i] = niitemperr(indata[i], erriter)
        else:
            tenii[i], nenii[i] = np.nan, np.nan
            teniierr[i], neniierr[i] = np.nan, np.nan

  tesiii = np.zeros(len(indata))
  nesiii = np.zeros(len(indata))
  tesiiierr = np.zeros(len(indata))
  nesiiierr = np.zeros(len(indata))
  for i in range(len(indata)):
    if indata[i]['BPT_NII'] == 0 and indata[i]['SIII9068_FLUX_CORR'] > err * indata[i]['SIII9068_FLUX_CORR_ERR'] and indata[i]['SII6730_FLUX_CORR'] > err * indata[i]['SII6730_FLUX_CORR_ERR'] and indata[i]['SII6716_FLUX_CORR'] > err * indata[i]['SII6716_FLUX_CORR_ERR'] and indata[i]['SIII6312_FLUX_CORR_REFIT'] > err * indata[i]['SIII6312_FLUX_CORR_REFIT_ERR']:
        tesiii[i], nesiii[i] = diags.getCrossTemDen(diag_tem = '[SIII] 6312/9069', diag_den = '[SII] 6731/6716', 
                                        value_tem = indata[i]['SIII6312_FLUX_CORR_REFIT']/indata[i]['SIII9068_FLUX_CORR'], 
                                              value_den = indata[i]['SII6730_FLUX_CORR']/indata[i]['SII6716_FLUX_CORR'], 
                                                    guess_tem = 10**4)
        tesiiierr[i], nesiiierr[i] = siiitemperr(indata[i], erriter)
    else: 
        tesiii[i], nesiii[i] = np.nan, np.nan
        tesiiierr[i], nesiiierr[i] = np.nan, np.nan


  teoiii = np.zeros(len(indata))
  teoiiierr = np.zeros(len(indata))
  for i in range(len(indata)):
        if tesiii[i] < 14000:
            teoiii[i] = 0.709 * tesiii[i] + 3610
            teoiiierr[i] = 0.709 * tesiiierr[i] + 3610
        else:
            teoiii[i], teoiiierr[i] = np.nan, np.nan

  indata.add_columns([tenii, tesiii, teoiii, nenii, nesiii, teniierr, tesiiierr, teoiiierr, neniierr, nesiiierr], 
                      names=('NII_TEMP', 'SIII_TEMP','OIII_TEMP', 'SII_DEN_NII', 'SII_DEN_SIII',
                            'NII_TEMP_ERR', 'SIII_TEMP_ERR','OIII_TEMP_ERR', 'SII_DEN_NII_ERR', 'SII_DEN_SIII_ERR'))

  return indata

In [ ]:
temdendata = dentemp(data[:5], 0, 100)

In [ ]:
temdendata

In [ ]:
icf = pn.ICF()
O1 = pn.Atom('O', 1)
O2 = pn.Atom('O', 2)
O3 = pn.Atom('O', 3)
icf_list = ['Ial06_16']

In [ ]:
pn.Atom('O')

In [ ]:
reg = 3

OII7319 = O2.getIonAbundance(int_ratio = temdendata[reg]['OII7319_FLUX_CORR_REFIT'], 
                             tem = temdendata[reg]['NII_TEMP'], den= temdendata[reg]['SII_DEN_NII'], 
                             to_eval='L(7320)', Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])
OII7330 = O2.getIonAbundance(int_ratio = temdendata[reg]['OII7330_FLUX_CORR_REFIT'], 
                             tem = temdendata[reg]['NII_TEMP'], den= temdendata[reg]['SII_DEN_NII'], 
                             to_eval='L(7330)', Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])
OIII5006 = O3.getIonAbundance(int_ratio = temdendata[reg]['OIII5006_FLUX_CORR'], 
                              tem = temdendata[reg]['OIII_TEMP'], den= temdendata[reg]['SII_DEN_SIII'], 
                              to_eval='L(5007)', Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])

OII7319, OII7330, OIII5006

In [ ]:
oii = O2.getIonAbundance(int_ratio = temdendata[reg]['OII7319_FLUX_CORR_REFIT']+temdendata[reg]['OII7330_FLUX_CORR_REFIT'], 
                             tem = temdendata[reg]['NII_TEMP'], den= temdendata[reg]['SII_DEN_NII'], 
                             to_eval='L(7320)+L(7330)', Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])
oii 

In [ ]:
np.mean((OII7319, OII7330))

In [ ]:
reg = 0

OII7319 = O2.getIonAbundance(int_ratio = temdendata[reg]['OII7319_FLUX_CORR_REFIT'], 
                             tem = temdendata[reg]['NII_TEMP'], den= temdendata[reg]['SII_DEN_NII'], 
                             wave = 7319.990, Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])
OII7330 = O2.getIonAbundance(int_ratio = temdendata[reg]['OII7330_FLUX_CORR_REFIT'], 
                             tem = temdendata[reg]['NII_TEMP'], den= temdendata[reg]['SII_DEN_NII'], 
                             wave = 7330.730, Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])
OIII5006 = O3.getIonAbundance(int_ratio = temdendata[reg]['OIII5006_FLUX_CORR'], 
                              tem = temdendata[reg]['OIII_TEMP'], den= temdendata[reg]['SII_DEN_SIII'], 
                              wave = 5006, Hbeta = temdendata[reg]['HB4861_FLUX_CORR'])

OII7319, OII7330, OIII5006

In [ ]:
def metal(indata, err, erriter):
    
    met7319 = np.zeros(len(indata))
    ion7319 = np.zeros(len(indata))
    ion5006 = np.zeros(len(indata))
    met7319err = np.zeros(len(indata))
    ion7319err = np.zeros(len(indata))
    ion5006err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7319_FLUX_CORR_REFIT'] > err * indata[j]['OII7319_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['NII_TEMP'] > err * indata[j]['NII_TEMP_ERR'] and \
        indata[j]['SII_DEN_NII'] > err * indata[j]['SII_DEN_NII_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR'] and\
        indata[j]['OIII_TEMP'] > err * indata[j]['OIII_TEMP_ERR'] and \
        indata[j]['SII_DEN_SIII'] > err * indata[j]['SII_DEN_SIII']:
            OII7319 = O2.getIonAbundance(int_ratio = indata[j]['OII7319_FLUX_CORR_REFIT'], tem = indata[j]['NII_TEMP'], 
                                         den= indata[j]['SII_DEN_NII'], wave = 7319.990, Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII5006 = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                          den= indata[j]['SII_DEN_SIII'], wave = 5006.843, Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII7319, 'O3': OIII5006}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met7319[j] = 12 + np.log10(abundance['Ial06_16'])
            ion7319[j] = OII7319
            ion5006[j] = OIII5006
            met7319err[j], ion7319err[j], ion5006err[j] = err7319(indata[j], erriter)
        else: 
            met7319[j], ion7319[j], ion5006[j] = np.nan, np.nan, np.nan
            met7319err[j], ion7319err[j], ion5006err[j] = np.nan, np.nan, np.nan
    
    met7330 = np.zeros(len(indata))
    ion7330 = np.zeros(len(indata))
    met7330err = np.zeros(len(indata))
    ion7330err = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7330_FLUX_CORR_REFIT'] > err * indata[j]['OII7330_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['NII_TEMP'] > err * indata[j]['NII_TEMP_ERR'] and \
        indata[j]['SII_DEN_NII'] > err * indata[j]['SII_DEN_NII_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR'] and\
        indata[j]['OIII_TEMP'] > err * indata[j]['OIII_TEMP_ERR'] and \
        indata[j]['SII_DEN_SIII'] > err * indata[j]['SII_DEN_SIII']:
            OII7330 = O2.getIonAbundance(int_ratio = indata[j]['OII7330_FLUX_CORR_REFIT'], tem = indata[j]['NII_TEMP'], 
                                         den= indata[j]['SII_DEN_NII'], wave = 7330.730, Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII5006 = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                          den= indata[j]['SII_DEN_SIII'], wave = 5006.843, Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII7330, 'O3': OIII5006}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met7330[j] = 12 + np.log10(abundance['Ial06_16'])
            ion7330[j] = OII7330
            met7330err[j], ion7330err[j] = err7330(indata[j], erriter)
        else: 
            met7330[j], ion7330[j] = np.nan, np.nan
            met7330err[j], ion7330err[j] = np.nan, np.nan
            
    met = np.zeros(len(indata))
    ionboth = np.zeros(len(indata))
    meterr = np.zeros(len(indata))
    ionbotherr = np.zeros(len(indata))
    
    for j in range(len(indata)):
        if indata[j]['OII7330_FLUX_CORR_REFIT'] > err * indata[j]['OII7330_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['OII7319_FLUX_CORR_REFIT'] > err * indata[j]['OII7319_FLUX_CORR_REFIT_ERR'] and \
        indata[j]['NII_TEMP'] > err * indata[j]['NII_TEMP_ERR'] and \
        indata[j]['SII_DEN_NII'] > err * indata[j]['SII_DEN_NII_ERR'] and \
        indata[j]['HB4861_FLUX_CORR'] > err * indata[j]['HB4861_FLUX_CORR'] and \
        indata[j]['OIII5006_FLUX_CORR'] > err * indata[j]['OIII5006_FLUX_CORR_ERR'] and\
        indata[j]['OIII_TEMP'] > err * indata[j]['OIII_TEMP_ERR'] and \
        indata[j]['SII_DEN_SIII'] > err * indata[j]['SII_DEN_SIII']:
            OII = O2.getIonAbundance(int_ratio = (indata[j]['OII7330_FLUX_CORR_REFIT']+indata[j]['OII7319_FLUX_CORR_REFIT']), 
                                         tem = indata[j]['NII_TEMP'], den= indata[j]['SII_DEN_NII'], 
                                         to_eval='L(7320)+L(7330)', Hbeta = indata[j]['HB4861_FLUX_CORR'])
            OIII = O3.getIonAbundance(int_ratio = indata[j]['OIII5006_FLUX_CORR'], tem = indata[j]['OIII_TEMP'], 
                                      den= indata[j]['SII_DEN_SIII'], to_eval='L(5007)', 
                                      Hbeta = indata[j]['HB4861_FLUX_CORR'])
            
            abunlist = {'O2': OII, 'O3': OIII}
            abundance = icf.getElemAbundance(abunlist, icf_list)
            met[j] = 12 + np.log10(abundance['Ial06_16'])
            ionboth[j] = OII
            meterr[j], ionbotherr[j] = errboth(indata[j], erriter)
        else: 
            met[j], ionboth[j] = np.nan, np.nan
            meterr[j], ionbotherr[j] = np.nan, np.nan
            
    indata.add_columns([met7319, met7330, ion7319, ion7330, ion5006, met7319err, met7330err, ion7319err,
                      ion7330err,ion5006err],
                     names=('MET_OII7319','MET_OII7330', 'OII7319_ABUN', 'OII7330_ABUN', 'OIII5006_ABUN',
                           'MET_OII7319_ERR','MET_OII7330_ERR', 'OII7319_ABUN_ERR', 'OII7330_ABUN_ERR', 'OIII5006_ABUN_ERR'))

    return indata

In [ ]:
def errboth(indata, iters):
    meterr = np.zeros(iters)
    ion2err = np.zeros(iters)
    ion3err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7319 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII = O2.getIonAbundance(int_ratio = (indata['OII7319_FLUX_CORR_REFIT']+noise7319)+(indata['OII7330_FLUX_CORR_REFIT']+noise7330), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), to_eval='L(7320)+L(7330)', 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), to_eval='L(5007)', 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII, 'O3': OIII}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        meterr[j] = 12 + np.log10(abundance['Ial06_16'])
        ion2err[j] = OII
        ion3err[j] = OIII
        
    meterr = np.array([meterr[i] for i in range(len(meterr)) if np.isnan(meterr[i]) == False])
    ion2err = np.array([ion2err[i] for i in range(len(ion2err)) if np.isnan(ion2err[i]) == False])
    ion3err = np.array([ion3err[i] for i in range(len(ion3err)) if np.isnan(ion3err[i]) == False])
    
    return np.std(meterr), np.std(ion2err), np.std(ion3err)


def err7319(indata, iters):
    met7319err = np.zeros(iters)
    ion7319err = np.zeros(iters)
    ion5006err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7319 = np.random.normal(0, indata['OII7319_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII7319 = O2.getIonAbundance(int_ratio = (indata['OII7319_FLUX_CORR_REFIT']+noise7319), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), wave = 7319, 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII5006 = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), wave = 5007, 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII7319, 'O3': OIII5006}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        met7319err[j] = 12 + np.log10(abundance['Ial06_16'])
        ion7319err[j] = OII7319
        ion5006err[j] = OIII5006
        
    met7319err = np.array([met7319err[i] for i in range(len(met7319err)) if np.isnan(met7319err[i]) == False])
    ion7319err = np.array([ion7319err[i] for i in range(len(ion7319err)) if np.isnan(ion7319err[i]) == False])
    ion5006err = np.array([ion5006err[i] for i in range(len(ion5006err)) if np.isnan(ion5006err[i]) == False])
    
    return np.std(met7319err), np.std(ion7319err), np.std(ion5006err)


def err7330(indata, iters):
    met7330err = np.zeros(iters)
    ion7330err = np.zeros(iters)
    
    for j in range(iters):
        
        noise7330 = np.random.normal(0, indata['OII7330_FLUX_CORR_REFIT_ERR'])
        noisetemp2 = np.random.normal(0, indata['NII_TEMP_ERR'])
        noiseden2 = np.random.normal(0, indata['SII_DEN_NII_ERR'])
        noise4861 = np.random.normal(0, indata['HB4861_FLUX_CORR_ERR'])
        noise5006 = np.random.normal(0, indata['OIII5006_FLUX_CORR_ERR'])
        noisetemp3 = np.random.normal(0, indata['OIII_TEMP_ERR'])
        noiseden3 = np.random.normal(0, indata['SII_DEN_SIII_ERR'])
        
        OII7330 = O2.getIonAbundance(int_ratio = (indata['OII7330_FLUX_CORR_REFIT']+noise7330), 
                                     tem = (indata['NII_TEMP']+noisetemp2), 
                                     den= (indata['SII_DEN_NII']+noiseden2), wave = 7330, 
                                     Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
        OIII5006 = O3.getIonAbundance(int_ratio = (indata['OIII5006_FLUX_CORR']+noise5006), 
                                      tem = (indata['OIII_TEMP']+noisetemp3), 
                                      den= (indata['SII_DEN_SIII']+noiseden3), wave = 5007, 
                                      Hbeta = (indata['HB4861_FLUX_CORR']+noise4861))
            
        abunlist = {'O2': OII7330, 'O3': OIII5006}
        abundance = icf.getElemAbundance(abunlist, icf_list)
        met7330err[j] = 12 + np.log10(abundance['Ial06_16'])
        ion7330err[j] = OII7330
        
    met7330err = np.array([met7330err[i] for i in range(len(met7330err)) if np.isnan(met7330err[i]) == False])
    ion7330err = np.array([ion7330err[i] for i in range(len(ion7330err)) if np.isnan(ion7330err[i]) == False])
    
    return np.std(met7330err), np.std(ion7330err)